In [1]:
import pandas as pd

In [2]:
# For distinct_categories: read the first column and rename it to "Category"
distinct_categories = pd.read_excel(r'..\acts_and_miracles\acts_and_miracles_preprep.xlsx', sheet_name='distinct_categories', header=0)
distinct_categories = distinct_categories.iloc[:, [0]]  # Get only the first column (column 1)
distinct_categories.columns = ['Category']

# For acts_and_miracles: read columns 1 and 3 (EventID and Category) where column 17 <= 9
acts_and_miracles_full = pd.read_excel(r'..\acts_and_miracles\acts_and_miracles_preprep.xlsx', sheet_name='acts_and_miracles', header=0)

# Filter rows where column 17 (index 16) has values <= 9
filtered_rows = acts_and_miracles_full.iloc[:, 16] <= 9
acts_and_miracles = acts_and_miracles_full.loc[filtered_rows, [acts_and_miracles_full.columns[0], acts_and_miracles_full.columns[2]]]
acts_and_miracles.columns = ['EventID', 'Category']

print(f"Loaded {len(distinct_categories)} distinct categories")
print(f"Loaded {len(acts_and_miracles)} filtered acts and miracles records")

Loaded 219 distinct categories
Loaded 678 filtered acts and miracles records


In [3]:
distinct_categories

,Category
0,Abundance
1,Anger
2,Annunciation
3,Anointing
4,Answer
...,...
214,Witness
215,Worship
216,Wrath
217,Wrestling


In [4]:
acts_and_miracles

,EventID,Category
0,1,Creation
1,2,Creation/Speech
2,3,Creation/Speech
3,4,Creation/Speech
4,5,Creation/Speech
...,...,...
673,688,Apparition/Prophecy/Judgment
674,689,Judgment/Prophecy/Death
675,690,Assurance/Promise
676,691,Recovery/Victory/Fulfilment


In [5]:
# Function to find matching EventIDs for each category
def get_matching_events(category):
    """
    Find all EventIDs where the category appears in the acts_and_miracles Category column
    (case-insensitive search)
    """
    if pd.isna(category):
        return ""
    
    # Case-insensitive search for the category in acts_and_miracles Category column
    matching_events = acts_and_miracles[
        acts_and_miracles['Category'].str.contains(str(category), case=False, na=False, regex=False)
    ]['EventID'].tolist()
    
    # Convert to comma-separated string
    return ','.join(map(str, matching_events))

# Create the new "Events" column
print("Processing categories and finding matching events...")
distinct_categories['Events'] = distinct_categories['Category'].apply(get_matching_events)

Processing categories and finding matching events...


In [6]:
distinct_categories

,Category,Events
0,Abundance,607
1,Anger,"500,653,672,675"
2,Annunciation,569
3,Anointing,"645,656,666,667"
4,Answer,"138,146,150,494,543,544,618"
...,...,...
214,Witness,654
215,Worship,"377,385"
216,Wrath,369
217,Wrestling,89


In [7]:
distinct_categories[distinct_categories.Events.str.contains('661')]

,Category,Events
56,Disobedience,"660,661"
57,Disowning,"661,662,668"
110,Judgment,"16,17,18,20,21,22,25,28,31,36,39,56,58,59,61,9..."
161,Rejection,"661,665"


In [8]:
distinct_categories[distinct_categories.Category.str.contains('Disobedie')]

,Category,Events
56,Disobedience,"660,661"


In [9]:
# Export to CSV
output_filename = 'distinct_categories_with_events.csv'
distinct_categories.to_csv(r'..\acts_and_miracles\distinct_categories_with_events.csv', index=False)

print(f"\nProcessing complete! File saved as '{output_filename}'")
print(f"\nDataFrame shape: {distinct_categories.shape}")
print("\nPreview of the result:")
print(distinct_categories.head(10))

# Show some statistics
non_empty_events = distinct_categories[distinct_categories['Events'] != '']['Events'].count()
print(f"\nCategories with matching events: {non_empty_events}")
print(f"Categories without matching events: {len(distinct_categories) - non_empty_events}")


Processing complete! File saved as 'distinct_categories_with_events.csv'

DataFrame shape: (219, 2)

Preview of the result:
          Category                                       Events
0        Abundance                                          607
1            Anger                              500,653,672,675
2     Annunciation                                          569
3        Anointing                              645,656,666,667
4           Answer                  138,146,150,494,543,544,618
5  Answered Prayer                      138,146,150,543,544,618
6       Apparition                                          688
7       Appearance                       92,160,163,266,311,438
8      Appointment  115,243,260,324,371,390,498,515,560,624,645
9    Assassination                                          510

Categories with matching events: 219
Categories without matching events: 0
